In [1]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import time
import itertools

from google.cloud import bigquery

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
def query_result(query):
    """
    Runs the query and returns the output. 
    
    Args:
        query (str): The query text
        
    Returns:
        (df): Output of the query as a dataframe
    """

    client = bigquery.Client('turing-230020')
    output = client.query(query).to_dataframe()
    client.close()
    
    return output

# Role Level Prediction

In [3]:
two_weeks_before_next_quarter = datetime.datetime(datetime.date.today().year, 3 * round((datetime.date.today().month - 1) // 3 + 1) + 1, 1).date() - timedelta(weeks=2)
two_weeks_before_next_quarter = two_weeks_before_next_quarter + timedelta(days=((0 - two_weeks_before_next_quarter.weekday() + 7) % 7))
two_weeks_before_previous_quarter = datetime.datetime(datetime.date.today().year, 3 * round((datetime.date.today().month - 1) // 3) + 1, 1).date() - timedelta(weeks=2)

In [4]:
train_start = datetime.date(2022, 1, 1)

if datetime.date.today() >= two_weeks_before_next_quarter:
  train_end = two_weeks_before_next_quarter
else:
  train_end = two_weeks_before_previous_quarter

demand_query = """
WITH 

position_data AS (
  SELECT 
    DISTINCT job_id, MAX (total_positions) AS positions
  FROM `turing-230020.curated.opps_positions` 
  WHERE 
    parent_opp_id = opportunityid
    AND job_id IS NOT NULL
  GROUP BY 1
)

SELECT
  DATE_TRUNC (DATE (j.created_date), WEEK(MONDAY)) AS month
, CASE 
    WHEN rt.role_type = 'Full Stack' THEN 'Full Stack'
    WHEN rt.role_type = 'Data Engineering' THEN 'Data Engineering'
    WHEN rt.role_type = 'DevOps' THEN 'DevOps'
    WHEN rt.role_type = 'Machine Learning' THEN 'Machine Learning'
    WHEN rt.role_type = 'Mobile' THEN 'Mobile'
    WHEN rt.role_type = 'UI/UX Developer' THEN 'UI/UX Developer'
    WHEN rt.role_type = 'QA' THEN 'QA'
    WHEN ((rt.role_type = 'Web Backend') OR (rt.role_type = 'Fullstack (BE-heavy)')) THEN 'Web Backend'
    WHEN ((rt.role_type = 'Cloud - AI/ML') OR (rt.role_type = 'Cloud - Data') OR (rt.role_type = 'Cloud - Solutions Architecture')) THEN 'Cloud'
    WHEN ((rt.role_type = 'Web Frontend') OR (rt.role_type = 'Fullstack (FE-heavy)')) THEN 'Web Frontend'
    ELSE 'Other'
  END AS role_type
, SUM (COALESCE (pd.positions, 1)) AS roles
FROM 
  turing-230020.raw.ms2_job j
  LEFT JOIN turing-230020.raw.ms2_job_role_type rt ON j.role_type_id = rt.id
  LEFT JOIN position_data pd ON j.id = pd.job_id
WHERE
  j.company <> 'OpenAI'
  AND j.id NOT IN (SELECT DISTINCT job_id FROM curated.test_jobs_master_table WHERE job_id IS NOT NULL)
GROUP BY 1, 2
ORDER BY 1 DESC, 2
"""

demand_data = query_result(demand_query)
demand_data = demand_data.rename(columns={'roles': 'y', 'month': 'date'})
demand_data = demand_data[((demand_data['date'] >= train_start) & (demand_data['date'] < train_end))]
demand_data['date'] = pd.to_datetime(demand_data['date'], format='%Y/%m/%d')
demand_data = demand_data.set_index('date')
demand_data = demand_data.sort_index()
demand_data

google.auth._default WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


,role_type,y
date,,
2022-01-03,Web Frontend,30
2022-01-03,Data Engineering,13
2022-01-03,DevOps,10
2022-01-03,Web Backend,20
2022-01-03,Mobile,9
...,...,...
2023-06-12,Machine Learning,2
2023-06-12,Full Stack,5
2023-06-12,DevOps,1


In [5]:
## Check - this keeps on changing. 

demand_data['y'].sum()

7780

In [6]:
forecast = {}
truth = {}
forecasted = 0
total = 0
coverage = 0
period = len(demand_data[demand_data['role_type'] == 'Full Stack'])

for role in list(demand_data['role_type'].unique()):
    forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 3
             )

    data = demand_data[demand_data['role_type'] == role][['y']]
    data = data.asfreq('W-MON')
    data = pd.DataFrame(
            data=data,
            index=pd.date_range(
                start=train_start,
                periods=period,
                freq=data.index.freq
            )
           )
    data = data.fillna(0)
    data_train = data.copy()
    data_train = data_train[data_train.index < pd.to_datetime(train_end)]

    forecaster.fit(y=data_train['y'])

    steps = 12
    predictions = forecaster.predict(steps=steps)

    print(role)
    print(f'Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})')
    print('Forecasted: ', int(predictions.sum()))

    forecasted += int(predictions.sum())
    forecast[role] = int(predictions.sum())

print('----------------------------------------------------------')
print('Forecasted Jobs: ', round (forecasted, 2))

Web Frontend
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  92
Data Engineering
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  43
DevOps
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  43
Web Backend
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  105
Mobile
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  40
Other
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  288
Full Stack
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  231
QA
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  28
Machine Learning
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  21
UI/UX Developer
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=76)
Forecasted:  6
Cloud
Train dates : 2022-01-03 00:00:00 --- 2023-06-12 00:00:00  (n=7

# Role-Skill Level Predicition

In [7]:
current_quarter = datetime.datetime(train_end.year, 3 * round((train_end.month - 1) // 3) + 1, 1).date()
current_quarter_week = current_quarter - timedelta(days=current_quarter.weekday())
previous_quarter_week = current_quarter_week - timedelta(weeks=24)
previous_quarter_week

datetime.date(2022, 10, 10)

In [8]:
job_requirements_query = """
WITH 

role_type AS (
  SELECT DISTINCT
    j.id AS job_id
  FROM 
    turing-230020.raw.ms2_job j
    LEFT JOIN turing-230020.raw.ms2_job_role_type rt ON j.role_type_id = rt.id
  WHERE
    j.created_date IS NOT NULL
    AND j.created_date >= '{}'
    AND j.created_date < '{}'
    AND j.company <> 'OpenAI'
    AND j.id NOT IN (SELECT DISTINCT job_id FROM curated.test_jobs_master_table WHERE job_id IS NOT NULL)
)

, job_continent_data AS ( 
  WITH 
    country_const_inc as (
      SELECT distinct cntr.job_id, tc.name 
      FROM raw.ms2_job_location_countries as cntr
      LEFT JOIN raw.tpm_countries as tc 
      On cntr.location_id = tc.id 
      where type = 'any'
      ORDER BY tc.name asc 
    ),

    country_const_exc as (
      SELECT distinct cntr.job_id, tc.name 
      FROM raw.ms2_job_location_countries as cntr
      LEFT JOIN raw.tpm_countries as tc 
      On cntr.location_id = tc.id 
      where type = 'none'
      ORDER BY tc.name asc 
    ),

    continent_const_inc as (
      SELECT distinct cont.job_id, tc.name 
      FROM raw.ms2_job_location_continents as cont
      LEFT JOIN raw.tpm_continents as tc
      ON cont.location_id = tc.id 
      where type = 'any'
      ORDER BY tc.name asc 
    ),

    continent_const_exc as (
      SELECT distinct cont.job_id, tc.name 
      FROM raw.ms2_job_location_continents as cont
      LEFT JOIN raw.tpm_continents as tc
      ON cont.location_id = tc.id 
      where type = 'none'
      ORDER BY tc.name asc 
    ),

    country_const_f_inc as (
      SELECT job_id, string_agg(name) as country_constraint_inc
      FROM country_const_inc
      GROUP BY job_id 
    ),

    continent_const_f_inc as (
      SELECT job_id, string_agg(name) as continent_constraint_inc
      FROM continent_const_inc
      GROUP BY job_id
    ),

    country_const_f_exc as (
      SELECT job_id, string_agg(name) as country_constraint_exc
      FROM country_const_exc
      GROUP BY job_id 
    ),

    continent_const_f_exc as (
      SELECT job_id, string_agg(name) as continent_constraint_exc
      FROM continent_const_exc
      GROUP BY job_id
    ),

    job_if_pathC as (

      SELECT msj.id,-- as job_id,
      MAX(CASE WHEN mjtc.type = 'Using devs found outside Turing' THEN 1 --'Path C'
      ELSE 0
      END) AS job_path_C,  

        FROM `turing-230020.raw.ms2_job` msj 
        LEFT JOIN `turing-230020.raw.ms2_job_type` msjt on msjt.job_id = msj.id
        LEFT JOIN `turing-230020.raw.ms2_job_type_catalog` mjtc on mjtc.id = msjt.job_type_catalog_id
        GROUP BY 1
    )
      
  SELECT 
    JB.id AS job_id,
    cntr_inc.country_constraint_inc as country_constraint_inc, 
    cntr_exc.country_constraint_exc as country_constraint_exc,
    cont_inc.continent_constraint_inc as continent_constraint_inc,
    cont_exc.continent_constraint_exc as continent_constraint_exc,
    CASE WHEN (op.individual_job_positions  IS NULL OR op.individual_job_positions  = 0) THEN 1 ELSE op.individual_job_positions  END AS total_positions,
    CASE WHEN (job_if_pathC.job_path_C IS NULL) THEN 0 ELSE job_if_pathC.job_path_C END AS Assigned_to_Strategic_Sourcing,
    CASE WHEN (COALESCE(cntr_inc.country_constraint_inc, cntr_exc.country_constraint_exc, cont_inc.continent_constraint_inc, cont_exc.continent_constraint_exc) IS NULL) THEN 0 ELSE 1 END AS Geo_Constraint,
    salesforce.curr_stage AS salesforce_status,
    COALESCE(op.parent_created_date, JB.created_date) AS job_creation_date, 



  FROM       `turing-230020.raw.ms2_job`       JB
  LEFT JOIN  country_const_f_inc as cntr_inc  ON JB.id = cntr_inc.job_id 
  LEFT JOIN  continent_const_f_inc as cont_inc  ON JB.id = cont_inc.job_id 
  LEFT JOIN  country_const_f_exc as cntr_exc  ON JB.id = cntr_exc.job_id 
  LEFT JOIN  continent_const_f_exc as cont_exc  ON JB.id = cont_exc.job_id
  LEFT JOIN  `turing-230020.matchingmetrics.opps_positions_draft` as op ON JB.id = op.job_id 
  LEFT JOIN  `turing-230020.curated.opportunity_value` salesforce ON (JB.salesforce_opportunity_id  = salesforce.opportunityid AND JB.id = salesforce.job_id)
  LEFT JOIN  job_if_pathC ON JB.id = job_if_pathC.id 

),

skill_challenge_map AS (
  SELECT
    skill_id,
    challenge_id
  FROM
    `raw.ms2_skill_challenges` dsm
    INNER JOIN `raw.dv2_challenge` c USING(challenge_id)
  -- WHERE
  --   active = 1
),
skill_vettable_name AS (
  SELECT 
  DISTINCT id, skill_name, CASE WHEN (skill_challenge_map.skill_id IS NOT NULL) THEN 1 ELSE 0 END AS Vettable,  

  FROM 
  `turing-230020.raw.base_all_skills_v4` bas
  LEFT JOIN skill_challenge_map ON (bas.id = skill_challenge_map.skill_id) 
),
job_skills_data_upto_5_layers AS (
  SELECT 
    job_id,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[0].skill'), NULL) AS INT64) skill_id,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[1].skill'), NULL) AS INT64) or_skill_id_1,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[2].skill'), NULL) AS INT64) or_skill_id_2,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[3].skill'), NULL) AS INT64) or_skill_id_3,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[4].skill'), NULL) AS INT64) or_skill_id_4,

  FROM 
    `turing-230020.raw.ms2_job_details`,
    UNNEST(JSON_EXTRACT_ARRAY(skills_expression, '$.children')) as child
  WHERE 
    JSON_EXTRACT_SCALAR(child, '$.operator') = 'or'
),

job_skills_with_or_condition AS (

  SELECT 
    jsd.* ,
    skill.skill_name AS Skill_Name,
    skill.Vettable AS Vettable, 

    or_skill1.skill_name AS or_Skill_Name_1,
    or_skill1.Vettable AS Vettable_1, 

    or_skill2.skill_name AS or_Skill_Name_2,
    or_skill2.Vettable AS Vettable_2, 

    or_skill3.skill_name AS or_Skill_Name_3,
    or_skill3.Vettable AS Vettable_3, 

    or_skill4.skill_name AS or_Skill_Name_4,
    or_skill4.Vettable AS Vettable_4, 


  FROM job_skills_data_upto_5_layers jsd
  LEFT JOIN skill_vettable_name     skill ON (    skill.id = jsd.skill_id) 
  LEFT JOIN skill_vettable_name or_skill1 ON (or_skill1.id = jsd.or_skill_id_1) 
  LEFT JOIN skill_vettable_name or_skill2 ON (or_skill2.id = jsd.or_skill_id_2) 
  LEFT JOIN skill_vettable_name or_skill3 ON (or_skill3.id = jsd.or_skill_id_3) 
  LEFT JOIN skill_vettable_name or_skill4 ON (or_skill4.id = jsd.or_skill_id_4) 

  WHERE job_id IN (
      SELECT DISTINCT job_id FROM role_type
    )
),


job_data_one_skill AS 
(
  SELECT 
    DISTINCT
    JB.id AS job_id, JB.company,JB.customer_category , 
    CASE WHEN (JB.must_have_total_years_of_experience IS NULL) THEN 0 ELSE 1 END AS YoE_Constraint,
    COALESCE(JB.must_have_total_years_of_experience, 0) AS must_have_YoE,
    CASE 
      WHEN (JB.must_have_total_years_of_experience < 3) THEN 'Less than 3 Years'
      WHEN (JB.must_have_total_years_of_experience >= 3 AND JB.must_have_total_years_of_experience < 6) THEN '3 to 6 Years'
      WHEN (JB.must_have_total_years_of_experience > 6) THEN 'More than 6 Years'
      ELSE 'Not Mentioned' 
    END AS Required_experience, 

    JBS.skill_id, JBS_LVL.name AS Level, 
    JB_STAT.id AS Job_Status_id , JB_STAT.status AS Job_Status,

    skill_vettable_name.skill_name AS Skill_Name,
    skill_vettable_name.Vettable AS Vettable, 

  FROM       `turing-230020.raw.ms2_job`       JB
  LEFT JOIN  `turing-230020.raw.ms2_job_skill` JBS ON JB.id = JBS.job_id
  LEFT JOIN  `turing-230020.raw.ms2_job_skill_level` JBS_LVL ON JBS.job_skill_level_id = JBS_LVL.id
  LEFT JOIN  `turing-230020.raw.ms2_job_status` JB_STAT ON JB.job_status_id = JB_STAT.id
  LEFT JOIN  skill_vettable_name ON JBS.skill_id = skill_vettable_name.id
  

  WHERE 
  job_id IN (
    SELECT DISTINCT job_id FROM role_type
  )
  AND JBS_LVL.name = 'required'
),

job_data AS (
  SELECT * 
  FROM job_data_one_skill
  INNER JOIN job_skills_with_or_condition js5l USING (skill_id, job_id, Skill_Name, Vettable)

  UNION ALL 
  
  SELECT * 
  FROM job_data_one_skill
  LEFT JOIN job_skills_with_or_condition js5l USING (skill_id, job_id, Skill_Name, Vettable)
  WHERE job_id NOT IN (
    SELECT DISTINCT job_id 
      FROM job_data_one_skill
      INNER JOIN job_skills_with_or_condition js5l USING (skill_id, job_id, Skill_Name, Vettable)
  )
)

, constraints_data AS (
  SELECT *, CONCAT ('[', ARRAY_TO_STRING([Skill_Name, or_Skill_Name_1, or_Skill_Name_2, or_Skill_Name_3, or_Skill_Name_4], ','), ']') AS or_skills
  FROM 
    job_data
    LEFT JOIN job_continent_data USING (job_id)
)

, required_constraints AS (
  SELECT
    job_id
  , MIN (country_constraint_inc) AS country_constraint_inc
  , MIN (country_constraint_exc) AS country_constraint_exc
  , MIN (continent_constraint_inc) AS continent_constraint_inc
  , MIN (continent_constraint_exc) AS continent_constraint_exc
  , MAX (must_have_YoE) AS must_have_YoE
  , STRING_AGG (or_skills, ',') AS all_skills
  FROM constraints_data
  GROUP BY 1
)

, position_data AS (
  SELECT 
    DISTINCT job_id, MAX (total_positions) AS positions
  FROM `turing-230020.curated.opps_positions` 
  WHERE 
    parent_opp_id = opportunityid
    AND job_id IS NOT NULL
  GROUP BY 1
)

, skill_level AS (
  SELECT
    j.id AS job_id
  , CASE 
      WHEN ((rt.role_type = 'Full Stack') OR (rt.role_type = 'Fullstack (BE-heavy)') OR (rt.role_type = 'Fullstack (FE-heavy)')) THEN 'Full Stack'
      WHEN rt.role_type = 'Data Engineering' THEN 'Data Engineering'
      WHEN rt.role_type = 'DevOps' THEN 'DevOps'
      WHEN rt.role_type = 'Machine Learning' THEN 'Machine Learning'
      WHEN rt.role_type = 'Mobile' THEN 'Mobile'
      WHEN rt.role_type = 'UI/UX Developer' THEN 'UI/UX Developer'
      WHEN rt.role_type = 'QA' THEN 'QA'
      WHEN rt.role_type = 'Web Backend' THEN 'Web Backend'
      WHEN ((rt.role_type = 'Cloud - AI/ML') OR (rt.role_type = 'Cloud - Data') OR (rt.role_type = 'Cloud - Solutions Architecture')) THEN 'Cloud'
      WHEN rt.role_type = 'Web Frontend' THEN 'Web Frontend'
      ELSE 'Other'
    END AS role_type
  , bas.skill_name
  , COALESCE (pd.positions, 1) AS positions
  FROM
  turing-230020.raw.ms2_job j
  LEFT JOIN turing-230020.raw.ms2_job_role_type rt ON j.role_type_id = rt.id
  LEFT JOIN turing-230020.raw.ms2_job_skill JBS ON JBS.job_id = j.id
  LEFT JOIN turing-230020.raw.base_all_skills_v4 bas ON JBS.skill_id = bas.id
  LEFT JOIN position_data pd ON j.id = pd.job_id
  WHERE
    j.id IN (SELECT DISTINCT job_id FROM role_type)
    AND JBS.job_skill_level_id = 1
)

SELECT DISTINCT
  sl.job_id
, COALESCE (sl.role_type, 'Other') AS role_type
, opportunity_created_date
, CASE 
    WHEN max_acceptable_rate = 0 THEN 40
    ELSE COALESCE (ov.max_acceptable_rate, 40) 
  END AS max_acceptable_rate
, rc.country_constraint_inc
, rc.country_constraint_exc
, rc.continent_constraint_inc
, rc.continent_constraint_exc
, rc.must_have_YoE
, rc.all_skills
, sl.positions
FROM
  skill_level sl
  LEFT JOIN required_constraints rc ON sl.job_id = rc.job_id
  LEFT JOIN turing-230020.curated.opportunity_value ov ON ov.job_id = sl.job_id
WHERE
  opportunity_created_date IS NOT NULL
""".format(previous_quarter_week.strftime('%Y-%m-%d'), train_end.strftime('%Y-%m-%d'))

job_requirements = query_result(job_requirements_query)
job_requirements

google.auth._default WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


,job_id,role_type,opportunity_created_date,max_acceptable_rate,country_constraint_inc,country_constraint_exc,continent_constraint_inc,continent_constraint_exc,must_have_YoE,all_skills,positions
0,10516,Mobile,2022-10-27 15:22:36+00:00,35.000000000,"Argentina,Colombia,Mexico",None,South America,None,4,"[iOS - Swift],[UX Design],[SwiftUI]",1
1,11058,Full Stack,2022-12-08 21:56:13+00:00,40.000000000,None,None,None,None,0,"[AWS],[PHP],[Laravel]",1
2,11664,DevOps,2023-01-17 15:46:26+00:00,47.000000000,None,None,South America,"Africa,Antarctica,Asia,Europe,North America,Oc...",4,"[Terraform],[Ansible],[Cloud]",6
3,13684,Other,2023-05-31 15:41:10+00:00,45.000000000,None,None,South America,None,3,"[Node.js],[GitHub],[Data Pipelines]",1
4,12655,DevOps,2023-03-13 13:17:37+00:00,47.000000000,None,None,South America,"Africa,Antarctica,Asia,Europe,North America,Oc...",0,"[Terraform],[Ansible],[Cloud]",1
...,...,...,...,...,...,...,...,...,...,...,...
1650,11689,Full Stack,2023-01-19 01:35:38+00:00,34.000000000,None,None,None,None,0,"[GraphQL],[Agile],[React]",2
1651,12211,Full Stack,2023-02-10 00:15:24+00:00,40.000000000,None,None,"North America,South America",None,5,"[React],[GraphQL],[Ruby on Rails]",1
1652,13697,Full Stack,2023-05-31 23:21:05+00:00,35.000000000,None,None,"North America,South America",None,5,"[React,Typescript]",1
1653,13554,Other,2023-05-23 07:36:51+00:00,23.000000000,None,None,Asia,None,6,"[C++],[Network Protocols],[TCP/IP]",1


In [9]:
def extractDigits(lst):
    res = []
    for el in lst:
        sub = el.split(', ')
        res.append(sub)
     
    return(res)


def dedup(k):
    k = sorted(k)
    dedup = list(k for k, _ in itertools.groupby(k))
    
    return dedup

skill_id_dict = {
      108: [108, 97, 448]
    , 567: [567, 2068]
    , 568: [568, 277]
    , 65: [65, 221]
    , 71: [1157, 71]
    , 257: [257, 443, 444, 467, 762, 189, 2140]
    , 70: [70, 60]
    , 351: [351, 394, 2084]
    , 541: [541, 204]
    , 1547: [1547, 460, 2114]
    , 1598: [1598, 483, 1571, 41]
    , 686: [686, 348, 347]
    , 25: [25, 2050]
    , 554: [554, 555]
    , 2096: [2096, 26]
    , 387: [387, 308]
    , 264: [264, 1408]
    , 358: [358, 1928, 2087]
    , 1821: [1821, 2061]
    , 114: [114, 86, 113, 2097]
    , 1938: [1696, 1123, 2150, 614]
    , 1025: [1025, 1389]
    , 127: [127, 162, 327]
    , 301: [301, 1991]
    , 1315: [1315, 464, 1675, 1676, 1258]
    , 2: [2, 3]
    , 2020: [2020, 401, 680]
    , 400: [400, 2032]
    , 223: [223, 1647]
    , 20: [20, 710, 1466]
    , 174: [174, 128]
    , 258: [258, 73, 1250]
    , 1286: [1286, 33, 1063, 812]
    , 1420: [1420, 1470]
    , 433: [433, 2133, 1827]
    , 449: [449, 1465]
    , 29: [29, 1688]
    , 166: [166, 173]
    , 107: [107, 309]
    , 125: [125, 1615, 111, 1059, 946, 328]
    , 2055: [2055, 425]
    , 93: [93, 598]
    , 2094: [2094, 256, 678, 2091, 2092, 1349, 1964, 397, 1300]
    , 2036: [2036, 1873, 1397, 2154, 1855]
    , 1939: [1939, 706, 1822, 153, 707, 1830]
}

def skill_dedup(or_skill):
    if (('Android SDK' in or_skill) | ('Android Studio' in or_skill)):
        return 'Android'
    if (('Android - Java' in or_skill) | ('Android/Java' in or_skill)):
        return 'Android: Java'
    if (('Kotlin' in or_skill) | ('Android/Kotlin' in or_skill)):
        return 'Android: Kotlin'
    if (('Spark' in or_skill)):
        return 'Apache Spark'
    if (('Confluence' in or_skill)):
        return 'Atlassian'
    if (('Business Application Development' in or_skill) | ('Business Intelligence' in or_skill) | ('Business Intelligence Engineering' in or_skill) | ('BI Support' in or_skill) | ('Tableau' in or_skill) | ('Power BI' in or_skill) | ('Looker' in or_skill)):  ## Edited
        return 'Business Intelligence (BI)'
    if (('Access' in or_skill)):
        return 'Microsoft Access'
    if (('Google Cloud Platform' in or_skill)):
        return 'GCP'
    if (('Data Modeling' in or_skill) | ('Data Analyst' in or_skill)):
        return 'Data Analysis'
    if (('Deep Learning Algorithms' in or_skill)):
        return 'Deep Learning'
    if (('Design Thinking' in or_skill) | ('Design Patterns' in or_skill) | ('Designer' in or_skill)):
        return 'Product Design'
    if (('Distributed Storage' in or_skill) | ('Distributed File Systems' in or_skill) | ('Distributed Systems' in or_skill)):
        return 'Distributed Programming'
    if (('Scrum' in or_skill) | ('Agile' in or_skill)):  ## Edited
        return 'Project Management'
    if (('Python(Django)' in or_skill)):
        return 'Django'
    if (('ERP Development' in or_skill)):
        return 'ERP'
    if (('Go/Golang' in or_skill)):
        return 'Go'
    if (('HTML5' in or_skill)):
        return 'HTML'
    if (('JSON Schema' in or_skill)):
        return 'JSON'
    if (('Lumen (for Laravel)' in or_skill) | ('PHP, Laravel' in or_skill)):
        return 'Laravel'
    if (('MS SQLServer' in or_skill)):
        return 'MS SQL Server'
    if (('MySQL' in or_skill) | ('PostgreSQL' in or_skill) | ('BigQuery' in or_skill)):  ## Edited
        return 'SQL'
    if (('UI Design' in or_skill) | ('webflow' in or_skill) | ('User-Centered Design' in or_skill) | ('Figma' in or_skill)):  ## Edited
        return 'UX Design'
    if (('Magento 2' in or_skill)):
        return 'Magento'
    if (('Open Source Frameworks' in or_skill) | ('Open Source Projects' in or_skill)):
        return 'Open Source'
    if (('Pentaho Kettle' in or_skill)):
        return 'Pentaho'
    if (('Salesforce Admin' in or_skill) | ('Salesforce Development' in or_skill) | ('Salesforce ' in or_skill) | ('Salesforce Lightning' in or_skill) | ('Salesforce Lightning Aura Components' in or_skill) ):  ## Edited
        return 'Salesforce'
    if (('Selenium WebDriver' in or_skill)):
        return 'Selenium'
    if (('Spring Framework' in or_skill) | ('Spring MVC' in or_skill)):
        return 'Spring'
    if (('SpringBoot' in or_skill)):
        return 'Spring Boot'
    if (('SwiftUI' in or_skill) | ('Swift' in or_skill)):
        return 'iOS - Swift'
    if (('Tensorflow 1' in or_skill)):
        return 'Tensorflow'
    if (('Python for Data Science' in or_skill) | ('Python Automation' in or_skill)):  ## Edited
        return 'Python'
    if (('Unix Development Environments' in or_skill)):
        return 'Unix'
    if (('GitHub' in or_skill) | ('GitLab' in or_skill)):  ## Edited
        return 'Git'
    if (('Windows' in or_skill) | ('Windows Applications' in or_skill) | ('Windows SDK' in or_skill) | ('Windows Services' in or_skill)):
        return 'Windows apps developement'
    if (('Yii 2 Framework' in or_skill)):
        return 'Yii Framework'
    if (('AWS Administration' in or_skill) | ('AWS Operations' in or_skill)):
        return 'AWS'
    if 'Crawlers' in or_skill:  ## Edited
        return 'WebScrape'
    if 'Java 8' in or_skill:
        return 'Java'
    if (('Linux Tools ' in or_skill) | ('Linux ' in or_skill)):  ## Edited
        return 'Linux'
    if 'CSS3' in or_skill:
        return 'CSS'
    if (('API Integrations  ' in or_skill) | ('REST/RESTful APIs' in or_skill) | ('Web API' in or_skill) | ('API Documentation' in or_skill) | ('Third party APIs' in or_skill)):
        return 'API Design'
    if 'Unity 3D' in or_skill:  ## Edited
        return 'Unity'
    if 'Javascript ES6' in or_skill:  ## Edited
        return 'JavaScript'
    if (('ETL' in or_skill) | ('Data Warehousing' in or_skill) | ('Data Extraction' in or_skill) | ('Data Engineering' in or_skill) | ('Data Pipelines' in or_skill) | ('Data Migration' in or_skill) | ('Data Spooling' in or_skill) | ('Data Cleansing' in or_skill)):
        return 'Data Engineer'
    if (('Angular 2+' in or_skill) | ('Angular 14' in or_skill) | ('AngularJS' in or_skill) | ('Angular CLI' in or_skill)):
        return 'Angular'
    if (('Automation Testing' in or_skill) | ('Functional Testing' in or_skill) | ('Automation Test Toolkit' in or_skill) | ('Android Testing' in or_skill) | ('Integration Testing' in or_skill) | ('System Integration Testing' in or_skill)):  ## Edited
        return 'Automation Testing'
    return or_skill


def skill_similar(string_skills, skills):
    check = []
    if (("['JavaScript']" in string_skills) & (('Node.js' in string_skills) | ('React' in string_skills) | ('Vue.js' in string_skills) | ('React Native' in string_skills) | ('Next.js' in string_skills) | ('Express.js' in string_skills) | ('React Hooks' in string_skills) | ('D3.js' in string_skills))):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('JavaScript')

    if (("['JavaScript']" in string_skills) & ('JavaScript Frameworks' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('JavaScript Frameworks')

    if (("['Ruby']" in string_skills) & ('Ruby on Rails' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Ruby')

    if (("['React']" in string_skills) & ('React Native' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('React Native')

    if (("['React']" in string_skills) & (('React Hooks' in string_skills) | ('Next.js' in string_skills))):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('React')

    if (("['Node.js']" in string_skills) & ('Express.js' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Node.js')

    if (("['Typescript']" in string_skills) & ('Angular' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Typescript')

    if (("['Python']" in string_skills) & ('Pandas' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Pandas')

    if (("['Python']" in string_skills) & (('Artificial Intelligence' in string_skills) | ('Machine Learning' in string_skills) | ('PyTorch' in string_skills))):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Python')
    
    if (("['Java']" in string_skills) & (('Spring' in string_skills) | ('Spring Boot' in string_skills) | ('JUnit' in string_skills))):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Java')
    
    if (("['Spring']" in string_skills) & ('Spring Boot' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('Spring')
    
    if (("['PHP']" in string_skills) & ('Laravel' in string_skills)):
        if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
        check.remove('PHP')

    if len(check) == 0:
            check = [element for innerList in skills for element in innerList]
    
    return extractDigits(check)


count_number = 3

def skill_combinations(df, count_number):
    combinations = []
    for _, row in df.iterrows():
        x = row['all_skills']
        skills = []
        count = 0
        if x is not None:
            all_skills = x.split(',')
            for and_skill in all_skills:
                or_skills = []
                for or_skill in and_skill.strip('][').split(', '):
                    count += 1
                    or_skills.append(skill_dedup(or_skill))
                skills.append(sorted(or_skills))
        
        string_skills = str(skills)
        skills = dedup(skills)
        skills = skill_similar(string_skills, skills)
        
        if count <= count_number:
            combinations.append(skills)

    return combinations

def correct_skills(x, combinations, count_number):
    # important_skills = []
    important_skills = ['Angular', 'React', 'Redux', 'Java', 'PHP', 'Vue.js', 'C++', 'Express.js'
                       , 'Python', 'Machine Learning', 'iOS Development', 'AWS', 'Artificial Intelligence'
                       , 'Node.js', 'SQL', 'Android: Java', 'Data Engineer', 'Typescript', 'C#'
                       , 'JavaScript', 'CSS', 'HTML', 'Ruby on Rails', 'Go/Golang', '.NET', 'MS SQL Server'
                       , 'Flutter', 'Data Science', 'iOS - Swift', 'Next.js', 'Android: Kotlin'
                       , 'Data Analysis', 'QA', 'Laravel', 'Symfony', 'CakePHP', 'Android'
                       , 'AWS Lambda', 'AWS Amplify', 'AWS Aurora', 'DevOps', 'Automation Testing', 'Cloud'
                       , 'Selenium', 'Azure', 'Airflow', 'Kubernetes', 'Ruby', 'Salesforce'
                       , 'Business Intelligence (BI)', 'JUnit', 'Spring Boot', 'Spring', 'UX Design'
                       , 'Project Management', 'Distributed Programming', 'WebScrape', 'Software Testing'
                       , 'React Hooks', 'Rust', 'Linux', 'Solution Architecture', 'GraphQL', 'Redis'
                       , 'API Design', 'System Design', 'Docker', 'Microservices', 'D3.js', 'Product Management']
    
    skills = []
    count = 0
    if x is not None:
        all_skills = x.split(',')
        for and_skill in all_skills:
            or_skills = []
            for or_skill in and_skill.strip('][').split(', '):
                count += 1
                or_skills.append(skill_dedup(or_skill))
            skills.append(sorted(or_skills))

        string_skills = str(skills)
        skills = dedup(skills)
        skills = skill_similar(string_skills, skills)

        count_important = 0
        for important_skill in important_skills:
            if important_skill in string_skills:
                count_important += 1
        
        if count >= 2:
            check = [element for innerList in skills for element in innerList]
            for combination in combinations:
                combiantion_set = [element for innerList in combination for element in innerList]
                if set(combiantion_set) <= set(check):
                    count_important_test = 0
                    for important_skill in important_skills:
                        if important_skill in combination:
                            count_important_test += 1
                    
                    if count_important_test == count_important:
                        skills = combination
                        break
        
        return str(sorted(skills))
    return x

forecast_skills = skill_combinations(job_requirements, count_number)
job_requirements['all_skills_sorted'] = job_requirements['all_skills'].apply(lambda x: correct_skills(x, forecast_skills, count_number))
job_requirements['week'] = job_requirements['opportunity_created_date'].apply(lambda x: (x - timedelta(days=x.weekday())).date())

In [10]:
dummy = ((job_requirements.groupby(['role_type', 'all_skills_sorted'])['positions'].sum() / job_requirements.groupby(['role_type'])['positions'].sum()) * 100).reset_index()
role_skill = dummy.rename(columns={'all_skills_sorted': 'skill_name', 'positions': 'percentage'})

In [11]:
forecast_df = pd.DataFrame(forecast.items(), columns=['role_type', 'forecast'])
forecast_df

,role_type,forecast
0,Web Frontend,92
1,Data Engineering,43
2,DevOps,43
3,Web Backend,105
4,Mobile,40
5,Other,288
6,Full Stack,231
7,QA,28
8,Machine Learning,21
9,UI/UX Developer,6


In [12]:
def count(skill_name):
    count = 0
    skills = []
    if skill_name is not None:
        all_skills = skill_name.split(',')
        for and_skill in all_skills:
            or_skills = []
            for or_skill in and_skill.strip('][').split(', '):
                count += 1
                or_skills.append(skill_dedup(or_skill))
            skills.append(sorted(or_skills))
        
    return count

role_skill_forecast = role_skill.merge(forecast_df, how='inner', on='role_type')
role_skill_forecast['role_skill_forecast'] = (role_skill_forecast['forecast'] * role_skill_forecast['percentage']) / 100
role_skill_forecast['count'] = role_skill_forecast['skill_name'].apply(lambda x: count(x))
role_skill_forecast

,role_type,skill_name,percentage,forecast,role_skill_forecast,count
0,Cloud,"[['API Design'], ['Artificial Intelligence']]",3.703704,10,0.37037,2
1,Cloud,"[['API Design'], ['Azure DevOps'], ['Kubernete...",3.703704,10,0.37037,3
2,Cloud,[['API Design']],3.703704,10,0.37037,1
3,Cloud,"[['AWS DevOps'], ['Amazon Redshift'], ['Big Da...",3.703704,10,0.37037,3
4,Cloud,"[['AWS Solutions Architecture'], ['Amazon Reds...",3.703704,10,0.37037,4
...,...,...,...,...,...,...
1116,Web Frontend,"[['SQL'], ['WebScrape']]",0.561798,92,0.516854,2
1117,Web Frontend,"[['Typescript'], ['Vue.js']]",0.561798,92,0.516854,2
1118,Web Frontend,[['UX Design']],0.561798,92,0.516854,1
1119,Web Frontend,"[['Vue.js'], ['WordPress']]",0.561798,92,0.516854,2


In [13]:
forecasted_skills_based = role_skill_forecast.groupby('skill_name')[['role_skill_forecast', 'count']].agg({'role_skill_forecast':'sum', 'count':'mean'}).reset_index()
forecasted_skills_based = forecasted_skills_based[((forecasted_skills_based['role_skill_forecast'] >= 1) & (forecasted_skills_based['count'] <= 4))]
forecasted_skills_based['skill_name']

3             [[' Laravel'], ['Angular'], ['Python']]
8                                      [[' Laravel']]
14                       [['.NET Core'], ['Angular']]
20     [['.NET Core'], ['MS SQL Server'], ['jQuery']]
28                            [['.NET'], ['Angular']]
                            ...                      
978                                   [['UX Design']]
979                                       [['Unity']]
981                                      [['Vue.js']]
982            [['iOS - Swift'], ['iOS Development']]
983                                 [['iOS - Swift']]
Name: skill_name, Length: 197, dtype: object

# Job - Skill DF

In [14]:
jd_query = """
SELECT 
  j.id
, COALESCE (jd.internal_job_description, j.notes) AS notes
FROM 
  raw.ms2_job j 
  LEFT JOIN raw.ms2_job_details jd ON j.id = jd.job_id
WHERE j.id IN {}
""".format(tuple(job_requirements['job_id']))

def skill_jd_count(jd, skill):
    if jd is None:
        return 0

    jd_check = jd.lower()
    count = 0
    all_skills = skill.split(',')
    for and_skill in all_skills:
        or_skills = []
        for or_skill in and_skill.strip('][').split(', '):
            check_skill = or_skill.lower()[1:-1]
            count += jd_check.count(check_skill)
    
    return count

In [15]:
jds = {}
jds_data = query_result(jd_query)
for skill in list(forecasted_skills_based['skill_name']):
    jd_score = {}
    for _, row in job_requirements.iterrows():
        if row['all_skills_sorted'] == skill:
            jd_score[str(row['job_id'])] = skill_jd_count(jds_data[jds_data['id'] == row['job_id']]['notes'].iloc[:].values[0], skill)
    jds[skill] = max(zip(jd_score.values(), jd_score.keys()))[1]

google.auth._default WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [16]:
job_skill_df = pd.DataFrame(jds.items(), columns=['Skill Tuple', 'Job ID'])
job_skill_df.head()

,Skill Tuple,Job ID
0,"[[' Laravel'], ['Angular'], ['Python']]",11662
1,[[' Laravel']],11060
2,"[['.NET Core'], ['Angular']]",10495
3,"[['.NET Core'], ['MS SQL Server'], ['jQuery']]",12104
4,"[['.NET'], ['Angular']]",13386


In [34]:
job_skill_df['Job ID'].nunique()

197

# Role - Skill Predictions

In [17]:
relative_role_skill_predictions = role_skill_forecast.copy()
relative_role_skill_predictions['role_skill_forecast'] = round (relative_role_skill_predictions['role_skill_forecast'])
relative_role_skill_predictions = relative_role_skill_predictions[relative_role_skill_predictions['role_skill_forecast'] >= 1]
total_sum = relative_role_skill_predictions['role_skill_forecast'].sum()
relative_role_skill_predictions['percentage'] = round ((relative_role_skill_predictions['role_skill_forecast'] / total_sum), 5)
relative_role_skill_predictions = relative_role_skill_predictions[['role_type', 'skill_name', 'percentage']].reset_index(drop=True)
relative_role_skill_predictions.head()

,role_type,skill_name,percentage
0,Cloud,"[['AWS'], ['Data Science'], ['Docker']]",0.00114
1,Cloud,"[['Machine Learning'], ['SQL']]",0.00114
2,Cloud,[['Machine Learning']],0.00114
3,Data Engineering,"[[' Laravel'], ['Angular'], ['Python']]",0.00228
4,Data Engineering,[['API Design']],0.00114


# Skill Tuples - Job Constraints

In [18]:
constraints_data = pd.DataFrame(columns=['Tuple', 'Dev Rate', 'Country Include', 'Country Exclude'
                                         , 'Continent Include', 'Continent Exclude', 'YoE Required'])
for skill in list(forecasted_skills_based['skill_name']):
    for _, row in job_requirements.iterrows():
        if row['all_skills_sorted'] == skill:
            constraints_data = constraints_data.append({'Tuple': row['all_skills_sorted'], 'Dev Rate': row['max_acceptable_rate']
                                                        , 'Country Include': row['country_constraint_inc'], 'Country Exclude': row['country_constraint_exc']
                                                        , 'Continent Include': row['continent_constraint_inc'], 'Continent Exclude': row['continent_constraint_exc']
                                                        , 'YoE Required': row['must_have_YoE']
                                                       }, ignore_index = True)

In [19]:
constraints_data_grouped = constraints_data.fillna('No')
constraints_data_grouped['Dev Rate'] = constraints_data_grouped['Dev Rate'].apply(lambda x: float(x))
constraints_data_grouped = constraints_data_grouped.groupby(['Tuple', 'Country Include', 'Country Exclude', 'Continent Include', 'Continent Exclude'])[['Dev Rate', 'YoE Required']].agg({'Dev Rate':['min', 'max', 'median'], 'YoE Required':['min', 'max', 'median', 'count']}).reset_index()
constraints_data_grouped.columns = constraints_data_grouped.columns.droplevel(1)
constraints_data_grouped.columns = ['Tuple', 'Country Include', 'Country Exclude', 'Continent Include', 'Continent Exclude'
                                    , 'Dev Rate Min', 'Dev Rate Max', 'Dev Rate Median', 'YoE Required Min'
                                    , 'YoE Required Max', 'YoE Required Median', 'Count']

constraints_data_count = constraints_data.groupby('Tuple')['Dev Rate'].count().reset_index()
constraints_data_count.columns = ['Tuple', 'Total Jobs']

constraints_data_with_percentages = constraints_data_grouped.merge(constraints_data_count, on='Tuple', how='left')
constraints_data_with_percentages['percent'] = (constraints_data_with_percentages['Count'] / constraints_data_with_percentages['Total Jobs']) * 100
constraints_data_with_percentages = constraints_data_with_percentages[constraints_data_with_percentages['percent'] >= 30]
constraints_data_with_percentages = constraints_data_with_percentages.merge(job_skill_df, left_on='Tuple', right_on='Skill Tuple', how='left')
constraints_data_with_percentages = constraints_data_with_percentages.drop(['Count', 'Total Jobs', 'Skill Tuple'], axis=1)
constraints_data_with_percentages.head()

,Tuple,Country Include,Country Exclude,Continent Include,Continent Exclude,Dev Rate Min,Dev Rate Max,Dev Rate Median,YoE Required Min,YoE Required Max,YoE Required Median,percent,Job ID
0,"[[' Laravel'], ['Angular'], ['Python']]",No,No,No,No,37.0,69.0,37.0,0,5,0.0,100.0,11662
1,[[' Laravel']],"Colombia,Mexico",No,No,"Africa,Asia,Europe,Oceania",24.0,24.0,24.0,0,0,0.0,50.0,11060
2,[[' Laravel']],No,No,No,No,40.0,40.0,40.0,0,0,0.0,50.0,11060
3,"[['.NET Core'], ['Angular']]",No,No,"Africa,Asia",No,26.0,40.0,33.0,3,4,3.5,100.0,10495
4,"[['.NET Core'], ['MS SQL Server'], ['jQuery']]",India,No,No,No,20.0,20.0,20.0,3,3,3.0,100.0,12104


In [33]:
constraints_data_with_percentages['Job ID'].nunique()

194

In [45]:
job_skill_df_u = constraints_data_with_percentages.copy()
import ast


def flatten_and_join(x):
    list_of_lists = ast.literal_eval(x)
    return '|'.join([item[0] for item in list_of_lists])

job_skill_df_u['Tuple'] = job_skill_df_u['Tuple'].apply(flatten_and_join)



job_skill_df_u.rename(columns = {'Tuple' : 'Skill_Tuple', 'Job ID':'job_id'}, inplace = True)
job_skill_df_u




,Skill_Tuple,Country Include,Country Exclude,Continent Include,Continent Exclude,Dev Rate Min,Dev Rate Max,Dev Rate Median,YoE Required Min,YoE Required Max,YoE Required Median,percent,job_id
0,Laravel|Angular|Python,No,No,No,No,37.0,69.0,37.0,0,5,0.0,100.000000,11662
1,Laravel,"Colombia,Mexico",No,No,"Africa,Asia,Europe,Oceania",24.0,24.0,24.0,0,0,0.0,50.000000,11060
2,Laravel,No,No,No,No,40.0,40.0,40.0,0,0,0.0,50.000000,11060
3,.NET Core|Angular,No,No,"Africa,Asia",No,26.0,40.0,33.0,3,4,3.5,100.000000,10495
4,.NET Core|MS SQL Server|jQuery,India,No,No,No,20.0,20.0,20.0,3,3,3.0,100.000000,12104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,UX Design,No,No,No,No,20.0,43.0,35.0,-1,5,0.0,83.333333,12885
241,Unity,No,No,No,No,13.0,40.0,34.5,0,0,0.0,100.000000,12416
242,Vue.js,No,No,No,No,25.0,40.0,40.0,0,4,4.0,100.000000,12414
243,iOS - Swift|iOS Development,No,No,No,No,20.0,34.0,31.5,0,4,0.0,100.000000,13374


In [59]:
job_skill_df.rename(columns = {'Skill Tuple' : 'Skill_Tuple', 'Job ID':'job_id'}, inplace = True)
job_skill_df_v = job_skill_df.copy()
job_skill_df_v = job_skill_df[~job_skill_df.job_id.isin(job_skill_df_u.job_id.unique())]

job_skill_df_v['Skill_Tuple'] = job_skill_df_v['Skill_Tuple'].apply(flatten_and_join)

job_skill_df_v = job_skill_df_u.append(job_skill_df_v)
job_skill_df_v


,Skill_Tuple,Country Include,Country Exclude,Continent Include,Continent Exclude,Dev Rate Min,Dev Rate Max,Dev Rate Median,YoE Required Min,YoE Required Max,YoE Required Median,percent,job_id
0,Laravel|Angular|Python,No,No,No,No,37.0,69.0,37.0,0.0,5.0,0.0,100.000000,11662
1,Laravel,"Colombia,Mexico",No,No,"Africa,Asia,Europe,Oceania",24.0,24.0,24.0,0.0,0.0,0.0,50.000000,11060
2,Laravel,No,No,No,No,40.0,40.0,40.0,0.0,0.0,0.0,50.000000,11060
3,.NET Core|Angular,No,No,"Africa,Asia",No,26.0,40.0,33.0,3.0,4.0,3.5,100.000000,10495
4,.NET Core|MS SQL Server|jQuery,India,No,No,No,20.0,20.0,20.0,3.0,3.0,3.0,100.000000,12104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,iOS - Swift|iOS Development,No,No,No,No,20.0,34.0,31.5,0.0,4.0,0.0,100.000000,13374
244,iOS - Swift,No,No,No,No,23.0,44.0,40.0,0.0,7.0,3.0,73.333333,10654
45,Android: Kotlin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14482
49,Angular|Node.js,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12985


In [60]:
import pandas_gbq
# Replace 'your_project_id', 'your_dataset_id', and 'your_table_id' with your actual values
project_id = 'turing-230020'
dataset_id = 'product_ds_supply'
table_id = 'forecasted_tuples'

# Insert the DataFrame into the BigQuery table

pandas_gbq.to_gbq(job_skill_df_v, f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

248 out of 248 rows loaded.<?, ?it/s]pandas_gbq.gbq INFO  
100%|██████████| 1/1 [00:00<00:00, 1092.55it/s]


Mapping

In [23]:
active_jobs_query = """

WITH 
skill_challenge_map AS (
  SELECT
    skill_id,
    challenge_id
  FROM
    `raw.ms2_skill_challenges` dsm
    INNER JOIN `raw.dv2_challenge` c USING(challenge_id)
  -- WHERE
  --   active = 1
),
skill_vettable_name AS (
  SELECT 
  DISTINCT id, skill_name, CASE WHEN (skill_challenge_map.skill_id IS NOT NULL) THEN 1 ELSE 0 END AS Vettable,  

  FROM 
  `turing-230020.raw.base_all_skills_v4` bas
  LEFT JOIN skill_challenge_map ON (bas.id = skill_challenge_map.skill_id) 
),
job_skills_data_upto_5_layers AS (
  SELECT 
    job_id,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[0].skill'), NULL) AS INT64) skill_id,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[1].skill'), NULL) AS INT64) or_skill_id_1,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[2].skill'), NULL) AS INT64) or_skill_id_2,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[3].skill'), NULL) AS INT64) or_skill_id_3,
    SAFE_CAST  (IFNULL(JSON_EXTRACT_SCALAR(child, '$.children[4].skill'), NULL) AS INT64) or_skill_id_4,

  FROM 
    `turing-230020.raw.ms2_job_details`,
    UNNEST(JSON_EXTRACT_ARRAY(skills_expression, '$.children')) as child
  WHERE 
    JSON_EXTRACT_SCALAR(child, '$.operator') = 'or'
),

job_skills_with_or_condition AS (

  SELECT 
    jsd.* ,
    skill.skill_name AS Skill_Name,
    skill.Vettable AS Vettable, 

    or_skill1.skill_name AS or_Skill_Name_1,
    or_skill1.Vettable AS Vettable_1, 

    or_skill2.skill_name AS or_Skill_Name_2,
    or_skill2.Vettable AS Vettable_2, 

    or_skill3.skill_name AS or_Skill_Name_3,
    or_skill3.Vettable AS Vettable_3, 

    or_skill4.skill_name AS or_Skill_Name_4,
    or_skill4.Vettable AS Vettable_4, 


  FROM job_skills_data_upto_5_layers jsd
  LEFT JOIN skill_vettable_name     skill ON (    skill.id = jsd.skill_id) 
  LEFT JOIN skill_vettable_name or_skill1 ON (or_skill1.id = jsd.or_skill_id_1) 
  LEFT JOIN skill_vettable_name or_skill2 ON (or_skill2.id = jsd.or_skill_id_2) 
  LEFT JOIN skill_vettable_name or_skill3 ON (or_skill3.id = jsd.or_skill_id_3) 
  LEFT JOIN skill_vettable_name or_skill4 ON (or_skill4.id = jsd.or_skill_id_4) 

  --WHERE job_id IN (      SELECT DISTINCT job_id FROM role_type )
),


job_data_one_skill AS 
(
  SELECT 
    DISTINCT
    JB.id AS job_id, JB.company,JB.customer_category , 
    CASE WHEN (JB.must_have_total_years_of_experience IS NULL) THEN 0 ELSE 1 END AS YoE_Constraint,
    COALESCE(JB.must_have_total_years_of_experience, 0) AS must_have_YoE,
    CASE 
      WHEN (JB.must_have_total_years_of_experience < 3) THEN 'Less than 3 Years'
      WHEN (JB.must_have_total_years_of_experience >= 3 AND JB.must_have_total_years_of_experience < 6) THEN '3 to 6 Years'
      WHEN (JB.must_have_total_years_of_experience > 6) THEN 'More than 6 Years'
      ELSE 'Not Mentioned' 
    END AS Required_experience, 

    JBS.skill_id, JBS_LVL.name AS Level, 
    JB_STAT.id AS Job_Status_id , JB_STAT.status AS Job_Status,

    skill_vettable_name.skill_name AS Skill_Name,
    skill_vettable_name.Vettable AS Vettable, 

  FROM       `turing-230020.raw.ms2_job`       JB
  LEFT JOIN  `turing-230020.raw.ms2_job_skill` JBS ON JB.id = JBS.job_id
  LEFT JOIN  `turing-230020.raw.ms2_job_skill_level` JBS_LVL ON JBS.job_skill_level_id = JBS_LVL.id
  LEFT JOIN  `turing-230020.raw.ms2_job_status` JB_STAT ON JB.job_status_id = JB_STAT.id
  LEFT JOIN  skill_vettable_name ON JBS.skill_id = skill_vettable_name.id
  

  --WHERE job_id IN (      SELECT DISTINCT job_id FROM role_type )
  WHERE JBS_LVL.name = 'required'
),

job_data AS (
  SELECT * 
  FROM job_data_one_skill
  INNER JOIN job_skills_with_or_condition js5l USING (skill_id, job_id, Skill_Name, Vettable)

  UNION ALL 
  
  SELECT * 
  FROM job_data_one_skill
  LEFT JOIN job_skills_with_or_condition js5l USING (skill_id, job_id, Skill_Name, Vettable)
  WHERE job_id NOT IN (
    SELECT DISTINCT job_id 
      FROM job_data_one_skill
      INNER JOIN job_skills_with_or_condition js5l USING (skill_id, job_id, Skill_Name, Vettable)
  )
)

, constraints_data AS (
  SELECT *, CONCAT ('[', ARRAY_TO_STRING([Skill_Name, or_Skill_Name_1, or_Skill_Name_2, or_Skill_Name_3, or_Skill_Name_4], ','), ']') AS or_skills
  FROM 
    job_data
)

SELECT
  job_id
, STRING_AGG (or_skills, ',') AS all_skills
FROM constraints_data
GROUP BY 1


"""

active_jobs = query_result(active_jobs_query)
active_jobs

google.auth._default WARNING No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


,job_id,all_skills
0,70,[React]
1,121,"[SQL],[Node.js],[React]"
2,6108,"[JavaScript],[React Native]"
3,8366,"[Python(Django)],[Node.js]"
4,6367,"[Python],[MySQL],[Node.js],[PostgreSQL]"
...,...,...
11946,13691,"[Node.js],[Typescript],[JavaScript]"
11947,13647,"[Node.js],[API Integrations ],[MySQL]"
11948,12985,"[Node.js],[Angular 2+],[Typescript]"
11949,12610,"[Microservices],[GraphQL],[Solution Architecture]"


In [24]:
#Condensation
active_jobs['all_skills_sorted'] = active_jobs['all_skills'].apply(lambda x: correct_skills(x, forecast_skills, count_number))


In [25]:
active_jobs_u = active_jobs[["job_id", "all_skills_sorted"]].copy()
active_jobs_u

,job_id,all_skills_sorted
0,70,[['React']]
1,121,"[['Node.js'], ['React'], ['SQL']]"
2,6108,[['React Native']]
3,8366,"[['Django'], ['Node.js']]"
4,6367,"[['Node.js'], ['Python'], ['SQL']]"
...,...,...
11946,13691,"[['Node.js'], ['Typescript']]"
11947,13647,"[['API Design'], ['Node.js'], ['SQL']]"
11948,12985,"[['Angular'], ['Node.js']]"
11949,12610,"[['GraphQL'], ['Microservices'], ['Solution Ar..."


In [26]:
import ast


def flatten_and_join(x):
    list_of_lists = ast.literal_eval(x)
    return '|'.join([item[0] for item in list_of_lists])

active_jobs_u['all_skills_sorted'] = active_jobs_u['all_skills_sorted'].apply(flatten_and_join)



active_jobs_u.rename(columns = {'all_skills_sorted' : 'Skill_Tuple'}, inplace = True)
active_jobs_u




,job_id,Skill_Tuple
0,70,React
1,121,Node.js|React|SQL
2,6108,React Native
3,8366,Django|Node.js
4,6367,Node.js|Python|SQL
...,...,...
11946,13691,Node.js|Typescript
11947,13647,API Design|Node.js|SQL
11948,12985,Angular|Node.js
11949,12610,GraphQL|Microservices|Solution Architecture


In [27]:
import pandas_gbq
# Replace 'your_project_id', 'your_dataset_id', and 'your_table_id' with your actual values
project_id = 'turing-230020'
dataset_id = 'product_ds_supply'
table_id = 'active_job_condensed_tuples'

# Insert the DataFrame into the BigQuery table

pandas_gbq.to_gbq(active_jobs_u, f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

11951 out of 11951 rows loaded.?it/s]pandas_gbq.gbq INFO  
100%|██████████| 1/1 [00:00<00:00, 327.27it/s]
